In [1]:
import struct
import pyDes
import binascii
from pathlib import Path

def get_decrypt(id):
    # For Python3, you'll need to use bytes, i.e.:
    #   data = b"Please encrypt my data"
    #   k = pyDes.des(b"DESCRYPT", pyDes.CBC, b"\0\0\0\0\0\0\0\0", pad=None, padmode=pyDes.PAD_PKCS5)
    k = pyDes.des("carwings", pyDes.ECB, IV=None, pad=None, padmode=pyDes.PAD_PKCS5)
    #print(type(id))
    eid = k.decrypt(id)
    #print("eid: {0}".format(eid))
    return eid
#print("Encrypted: %r" % binascii.hexlify(enc_tcuid))
#print("Decrypted: %r" % binascii.hexlify(k.decrypt(enc_tcuid)))
#get_bits = lambda bits, value:  get_bits(bits -1, value) +  (0x01&(value >> bits-1))  if bits > 0 else 0
get_bits = lambda value: get_bits(value>>1) +  (0x01&(value))  if  value.bit_length() > 0 else 0
def unpack_sxmhead(sxm_bin, sxm_out, mapv):
    sxm_head32 = '>H10s16s16sHHcHHH32sc17sH'
    sxm_head64 = '>H10s16s16sHHcHHH64sc17sH'
    if mapv == 32:
        #   header_size  send_file_no      label_x02
        head = struct.unpack_from(sxm_head32, sxm_bin, 0)
    else:
        head = struct.unpack_from(sxm_head64, sxm_bin, 0)
    print("head_size: {0}".format(head[0]))
    print("Ownerid: {0}".format(head[1]))
    print("tcuid: {0}".format(get_decrypt(head[2])))
    print("vhid: {0}".format(get_decrypt(head[3])))
    print("fileno: {0}".format(head[4]))
    print("softvid: {0} softv: {1}".format(head[5], head[6]))
    print("confvid: {0} confv: {1}".format(head[7], head[8]))
    print("mapvid: {0} mapv: {1}".format(head[9], head[10]))
    print("flag: {0}".format(head[11]))
    print("vin: {0}".format(head[12]))
    print("record_num: {0}".format(head[13]))
    return (head[0], head[13])

def unpack_sxmbody(sxm_bin, sxm_out, head_size, rec_num):
    sxm_dict = {
        "0x0001":[">12s", 1],
        "0x0005":[">6s", 1],
        "0x0006":[">9s", 1],
        "0x0007":[">3s", 1],
        "0x0008":[">3s", 1],
        "0x000C":[">8s", 1],
        "0x000D":[">2s", 1],
        "0x000E":[">2s", 1],
        "0x000F":[">4s", 1],
        "0x0012":[">1s", 1],
        "0x002C":[">4s", 1],
        "0x0040":[">9s", 1],
        "0x0041":[">9s", 1],
        "0x0080":[">7s", 1],
        "0x0081":[">7s", 1],
        "0x0082":[">8s", 1],
        "0x0083":[">8s", 1],
        "0x0084":[">4s", 1],
        "0x0085":[">4s", 1],
        "0x0086":[">2s", 1],
        "0x0087":[">2s", 1],
        "0x0089":[">2s", 1],
        "0x008B":[">2s", 1],
        "0x008C":[">2s", 1],
        "0x008E":[">1s", 1],
        "0x008F":[">1s", 1],
        "0x0090":[">1s", 1],
        "0x0091":[">1s", 1],
        "0x0092":[">2s", 1],
        "0x0093":[">2s", 1],
        "0x0095":[">16s", 1],
        "0x0096":[">2s", 1],
        "0x0097":[">3s", 1],
        "0x0098":[">4s", 1],
        "0x0099":[">2s", 1],
        "0x009A":[">2s", 1],
        "0x009B":[">2s", 1],
        "0x009F":[">2s", 1],
        "0x00A0":[">3s", 1],
        "0x00A1":[">3s", 1],
        "0x00A2":[">4s", 1],
        "0x00A3":[">2s", 1],
        "0x00A4":[">4s", 1],
        "0x00A5":[">2s", 1],
        "0x00A8":[">20s", 1],
        "0x00AA":[">2s", 1],
        "0x00AB":[">2s", 1],
        "0x00AC":[">2s", 1],
        "0x00AD":[">2s", 1],
        "0x00AE":[">2s", 1],
        "0x00AF":[">2s", 1],
        "0x00B1":[">46s", 1],
        "0x00B4":[">70s", 1],
        "0x00B7":[">1s", 1],
        "0x00B8":[">25s", 1],
        "0x00B9":[">24s", 1],
        "0x00BA":[">4s", 1],
        "0x00BD":[">9s", 1],
        "0x00BE":[">13s", 1],
        "0x00BF":[">1s", 1],
        "0x00C4":[">8s", 2],
        "0x00C5":[">22s", 0],
        "0x00CC":[">7s", 1],
        "0x00CD":[">1s", 1],
        "0x00CE":[">2s", 1],
        "0x00CF":[">8s", 1],
        "0x00D0":[">1s", 1],
        "0x00D1":[">2s", 1],
        "0x00D2":[">2s", 1],
        "0x00D3":[">18s", 0],
        "0x00D4":[">18s", 0],
        "0x00D5":[">18s", 0],
        "0x00D6":[">18s", 0],
        "0x00D7":[">10s", 1],
        "0x00D8":[">2s", 1],
        "0x00D9":[">2s", 1],
        "0x00DA":[">42s", 1],
        "0x00DE":[">18s", 1],
        "0x00DF":[">2s", 1],
        "0x00EA":[">4s", 1],
        "0x00EE":[">2s", 1],
        "0x00F8":[">39s", 0],
        "0x00FA":[">1s", 1],
        "0x0101":[">301s", 1],
        "0x0103":[">1312s", 1],
        "0x0108":[">107s", 1],
        "0x0109":[">311s", 1],
        "0x010A":[">12s", 1],
        "0x010B":[">200s", 1],
        "0x010C":[">195s", 1],
        "0x010F":[">254s", 1],
        "0x0110":[">1s", 1],
        "0x0111":[">4s", 1],
        "0x0112":[">2s", 1],
        "0x0113":[">2s", 1],
        "0x0114":[">2s", 1],
        "0x0115":[">2s", 1],
        "0x0116":[">2s", 1],
        "0x0117":[">46s", 1],
        "0x0118":[">46s", 1],
        "0x0119":[">40s", 1],
        "0x011A":[">40s", 1],
        "0x011B":[">2s", 1],
        "0x011C":[">2s", 1],
        "0x011D":[">46s", 1],
        "0x011E":[">46s", 1],
        "0x011F":[">40s", 1],
        "0x0120":[">40s", 1],
        "0x0121":[">14s", 1],
        "0x0122":[">4s", 1],
        "0x0123":[">4s", 1],
        "0x0124":[">2s", 1],
        "0x0125":[">4s", 1],
        "0x0126":[">15s", 1],
        "0x0127":[">4s", 1],
        "0x0128":[">6s", 1],
        "0x0129":[">4s", 0],
        "0x012A":[">4s", 0],
        "0x012B":[">4s", 0],
        "0x012C":[">4s", 0],
        "0x012D":[">4s", 0],
        "0x012E":[">4s", 0],
        "0x012F":[">4s", 0],
        "0x0130":[">4s", 0],
        "0x0131":[">4s", 0],
        "0x0132":[">4s", 0],
        "0x0134":[">2s", 1],
        "0x0135":[">1s", 1],
    }
    offset = head_size + 2
    for i in range(rec_num):
        data_size = struct.unpack_from(">H", sxm_bin, offset)[0]
        print("data_size:{0} record_num: {1} loop:{2}".format(data_size, rec_num, i))
        offset += 2
        read_size = 0
        old_offset = offset
        gps_data = struct.unpack_from(">10s", sxm_bin, offset)[0]
        print("GPS:{0}".format(gps_data))
        offset += 10
        while read_size < data_size:
            label = struct.unpack_from(">H", sxm_bin, offset)[0]
            offset += 2
            label_inf = sxm_dict["0x{:04X}".format(label)]
            if label_inf[1] == 1:#fix length
                label_data = struct.unpack_from(label_inf[0], sxm_bin, offset)[0]
                offset += int(label_inf[0][1:-1])
            elif label_inf[1] == 0:#valuable
                label_data_num = struct.unpack_from(">b", sxm_bin, offset)[0]
                offset += 1
                if label == 0x00F8:
                    label_data_1 = struct.unpack_from(">15s", sxm_bin, offset)[0]
                    offset += 15
                    i_flg = struct.unpack_from(">H", sxm_bin, offset)[0]
                    offset += 2
                    w_flg = struct.unpack_from(">H", sxm_bin, offset)[0]
                    offset += 2
                    isize = get_bits(i_flg)*4
                    print("isize:{}".format(isize))
                    i_data = struct.unpack_from(">{0}s".format(isize), sxm_bin, offset)[0]
                    offset += isize
                    wsize = get_bits(w_flg)
                    print("wsize:{}".format(wsize))
                    w_data = struct.unpack_from(">{0}s".format(wsize), sxm_bin, offset)[0]
                    offset += wsize
                    label_data = label_data_1 + bytes(i_flg) + bytes(w_flg) + i_data + w_data
                else:
                    ssize = int(label_inf[0][1:-1])*label_data_num
                    print("label:{:04X} loop:{}".format(label, label_data_num))
                    label_data = struct.unpack_from(">{0}s".format(ssize), sxm_bin, offset)[0]
                    offset += ssize
            elif label_inf[1] == 2:#sp
                if label[0] == 0x00c4:
                    label_data_1 = struct.unpack_from(lable_inf[0], sxm_bin, offset)[0]
                    offset += int(label_inf[0][1:-1])
                    ssize = int(label_data_1[-1])
                    label_data_2 = struct.unpack_from(">{0}s".format(ssize), sxm_bin, offset)[0]
                    offset += sszie
                    label_data = label_data_1 + label_data_2
            read_size = offset - old_offset
            print("label:{:04X} value:{}".format(label, label_data))
        if read_size != data_size:
            print("data_size over: {0} {1}".format(data_size, read_size))
        else:
            print("read Next recode from: {:04X}\n".format(offset))
                    
def unpack(input_f, output_f, mapv):
    with open(input_f, 'rb') as in_f:
        with open(output_f, 'w') as ou_f:
            data = in_f.read()
            (head_size, rec_num) = unpack_sxmhead(data, ou_f, mapv)
            unpack_sxmbody(data, ou_f, head_size, rec_num)
#unpack("sxm_data_v", "sxm_txt", 32)
#unpack("Probe_1_ABCDEFGHIJK000001_55D62ECF31C04999538FFB9D4D0E93191FACE0009E5AAC57_01_01_01_2017020700005671701_EV.OK", "sxm_txt", 32)
p = Path("./SXM")

# dir_A直下のファイルとディレクトリを取得
# Path.glob(pattern)はジェネレータを返す。結果を明示するためlist化しているが、普段は不要。
for f in list(p.glob("*.NG")):
    print("read file: {}".format(f))
    unpack(f, "sxm_txt", 32)

In [18]:
if (7 & 0x80) > 1:
    print(1)
else:
    print(0)

0
